In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 51.5 MB/s 
     |████████████████████████████████| 636 kB 62.4 MB/s 
     |████████████████████████████████| 50 kB 8.8 MB/s 
     |████████████████████████████████| 895 kB 61.9 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%matplotlib inline
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import classification_report
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW# get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import time

!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/utils.py .
from utils import *
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/Custom_Dataset_Class.py .
from Custom_Dataset_Class import CustomDataset
!cp drive/MyDrive/Colab\ Notebooks/MSc-Individual-Project/pytorchtools.py .
from pytorchtools import EarlyStopping
#from Bert_Classification import Bert_Classification_Model
#from RoBERT import RoBERT_Model

#from BERT_Hierarchical import BERT_Hierarchical_Model
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer

In [4]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [6]:
#change to where you store mimic3 data
MIMIC_3_DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/datasets'

train_df = pd.read_csv('%s/train_50.csv' % MIMIC_3_DIR)
eval_df = pd.read_csv('%s/dev_50.csv' % MIMIC_3_DIR)
test_df = pd.read_csv('%s/test_50.csv' % MIMIC_3_DIR)

train_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,23657,125544,Physician,title,584.9;403.91;311,1
1,30845,115327,Nursing/other,addendum,276.0;584.9;287.5;518.81;428.0,1
2,81475,101662,Physician,title,250.00;785.52;427.31;995.92;530.81;584.5;038.9...,1
3,70220,163685,Nursing,dnr,511.9;276.0;584.9;995.92;414.01;285.1;038.9;51...,1
4,5331,142049,Nursing/other,hypotension,250.00;401.9;427.31;530.81;414.01,1


In [7]:
full_df = pd.concat([train_df, eval_df, test_df], ignore_index=True)

In [8]:
# split labels by ";", then convert to list
def split_lab (x):
    #print(x)
    return x.split(";")

full_df['LABELS'] = full_df['LABELS'].apply(split_lab)
#full_df['TEXT'] = full_df['TEXT'].apply(split_lab)

full_df.head()

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,LABELS,length
0,23657,125544,Physician,title,"[584.9, 403.91, 311]",1
1,30845,115327,Nursing/other,addendum,"[276.0, 584.9, 287.5, 518.81, 428.0]",1
2,81475,101662,Physician,title,"[250.00, 785.52, 427.31, 995.92, 530.81, 584.5...",1
3,70220,163685,Nursing,dnr,"[511.9, 276.0, 584.9, 995.92, 414.01, 285.1, 0...",1
4,5331,142049,Nursing/other,hypotension,"[250.00, 401.9, 427.31, 530.81, 414.01]",1


In [9]:
#load multi label binarizer for one-hot encoding
mlb = MultiLabelBinarizer(sparse_output=True)

#labels_onehot = mlb.fit_transform(train_df.pop('LABELS'))
#labels_onehot[0][1]

In [10]:
#change label to one-hot encoding per code
full_df = full_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(full_df.pop('LABELS')),
                columns=mlb.classes_))

full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61
0,23657,125544,Physician,title,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30845,115327,Nursing/other,addendum,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,81475,101662,Physician,title,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,70220,163685,Nursing,dnr,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,5331,142049,Nursing/other,hypotension,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373477,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373478,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373479,91939,149837,Discharge summary,admission date discharge date date of birth se...,5630,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373480,96777,176399,Discharge summary,admission date discharge date date of birth se...,5890,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
full_df.HADM_ID.unique().shape

(11021,)

In [12]:
# Convert columns to list of one hot encoding
icd_classes_50 = mlb.classes_

full_df['labels'] = full_df[icd_classes_50].values.tolist()
#train_df.sort_values(['length'], ascending=False, inplace=True)
full_df.head()


,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61,labels
0,23657,125544,Physician,title,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,30845,115327,Nursing/other,addendum,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
2,81475,101662,Physician,title,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,70220,163685,Nursing,dnr,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,5331,142049,Nursing/other,hypotension,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [13]:
full_df = full_df.drop(full_df[full_df['length']<300].index)

In [14]:
full_df

,SUBJECT_ID,HADM_ID,CATEGORY,TEXT,length,008.45,038.9,244.9,250.00,272.0,272.4,276.0,276.1,276.2,285.1,285.9,287.5,305.1,311,401.9,403.90,403.91,410.71,412,414.01,427.31,428.0,486,496,507.0,511.9,518.0,518.5,518.81,530.81,584.5,584.9,585.9,599.0,747.0,769,770.81,774.2,776.6,779.3,779.81,785.52,995.92,V05.3,V29.0,V30.00,V30.01,V31.01,V45.81,V58.61,labels
220400,605,115545,Nursing,pt is an yo woman with uti complicated by sept...,300,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
220401,1766,185657,Nursing/other,t sicu npn review of systems neuro alert orien...,300,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
220402,29348,127366,Nursing/other,condition update see carevue for specifics rem...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
220403,18447,159812,Nursing/other,tsicu npn o ros neuro sedated on propofol w pr...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
220404,2468,133778,Nursing/other,npn bili resp pt remains on simv with settings...,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373477,93623,187232,Discharge summary,admission date discharge date date of birth se...,5171,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
373478,96260,110058,Discharge summary,admission date discharge date date of birth se...,5173,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
373479,91939,149837,Discharge summary,admission date discharge date date of birth se...,5630,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
373480,96777,176399,Discharge summary,admission date discharge date date of birth se...,5890,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [15]:
train_df, test_df = train_test_split(full_df, test_size=0.2)

In [16]:
train_df, dev_df = train_test_split(train_df, test_size=0.2)

In [17]:
train_df.sort_values(['length'], inplace=True)
dev_df.sort_values(['length'], inplace=True)
test_df.sort_values(['length'], inplace=True)


In [18]:
#convert into 2 columns dataframe
train_df = pd.DataFrame(train_df, columns=['TEXT', 'labels'])
train_df.columns=['text', 'labels']
train_df.head()

dev_df = pd.DataFrame(dev_df, columns=['HADM_ID', 'TEXT', 'labels'])
dev_df.columns=['id', 'text', 'labels']
dev_df.head()


test_df = pd.DataFrame(test_df, columns=['HADM_ID', 'TEXT', 'labels'])
test_df.columns=['id', 'text', 'labels']
test_df.head()

,id,text,labels
220413,141118,cardiac arrest assessment neuro exam as below ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
354037,175706,subjective patient intubated and sedated objec...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
220537,103143,npn o infant continues on conventional vent se...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
220605,155981,this is a year old female pt with h o breast c...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
354038,131717,patient test information indication left ventr...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [19]:
train_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df.head()

,id,text,labels
0,141118,cardiac arrest assessment neuro exam as below ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
1,175706,subjective patient intubated and sedated objec...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,103143,npn o infant continues on conventional vent se...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,155981,this is a year old female pt with h o breast c...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,131717,patient test information indication left ventr...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [20]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        '''
            Load Pretrained model here
            Use return_dict=False for compatibility for 4.x

        '''
        self.l1 = transformers.AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", return_dict=False)
        #self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)


        self.l2 = torch.nn.Dropout(0.3)

        '''
            Changed Linear Output layer to 50 based on the class
        '''
        self.l3 = torch.nn.Linear(768, 50)

    def forward(self, ids, mask, token_type_ids):
#        print("ids: ", ids.size(), "mask: ", mask.size(), "token type ids: ", token_type_ids.size())
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [21]:
# Defining some key variables to configure model training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 3e-05

#set tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

#custom dataset for BERT class
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        
        '''
            set text as training data
            set labels as targets
        '''
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }



Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [22]:
#load df to dataset

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_set = CustomDataset(dev_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [23]:
#data loader
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False
                }

training_loader = DataLoader(training_set, **train_params)
valid_loader = DataLoader(valid_set, **val_params)
testing_loader = DataLoader(testing_set, **test_params)

In [24]:
#loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

#optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [25]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

In [26]:
# Evaluate the model

def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    print(f'Epoch: {epoch}, Validation Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [27]:
start_epoch=5
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'
resume = True     
if resume:
    if os.path.isfile(f"%s/models/model_allnotesshuffle50_epoch{start_epoch}.pth" % DIR):
        print("Resume from checkpoint...")
        checkpoint = torch.load(f"%s/models/model_allnotesshuffle50_epoch{start_epoch}.pth" % DIR)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        initepoch = checkpoint['epoch']
        print("====>loaded checkpoint (epoch{})".format(checkpoint['epoch']))
    else:
        print("====>no checkpoint found.")
        initepoch = 0

#patience = 3
#early_stopping = EarlyStopping(patience, verbose=True)


for epoch in tqdm(range(EPOCHS)):
    train(epoch)
    validation(epoch)

    if (epoch+start_epoch+1)%5 == 0:
        checkpoint = {"model_state_dict": model.state_dict(),
                      "optimizer_state_dict": optimizer.state_dict(),
                      "epoch": epoch+start_epoch+1}
        path_checkpoint = f"%s/models/model_allnotesshuffle50_epoch{epoch+start_epoch+1}.pth" % DIR
        torch.save(checkpoint, path_checkpoint)

#

Resume from checkpoint...


KeyboardInterrupt: ignored

In [28]:
DIR = '/content/drive/MyDrive/Colab Notebooks/MSc-Individual-Project/'

checkpoint = torch.load(f"%s/models/model_allnotesshuffle50_epoch10.pth" % DIR)
model.load_state_dict(checkpoint['model_state_dict'])



<All keys matched successfully>

In [31]:
# Evaluate the model

def evaluation():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [32]:
dev_out, dev_tar, losses = evaluation()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Loss:  0.14


### Normal evaluation

In [33]:
outputs = np.array(dev_out) >= 0.5
targets = dev_tar
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.7828401713425515
F1 Score (Macro) = 0.7124172893276609


In [34]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.7714    0.7631    0.7672       840
       038.9     0.8315    0.8088    0.8200      2385
       244.9     0.8391    0.7802    0.8085      1965
      250.00     0.8417    0.7220    0.7773      2953
       272.0     0.8624    0.5698    0.6862      1397
       272.4     0.8498    0.7108    0.7741      2849
       276.0     0.6853    0.8629    0.7639      1933
       276.1     0.8307    0.7146    0.7683      1566
       276.2     0.8173    0.7803    0.7984      2517
       285.1     0.7091    0.8229    0.7618      1982
       285.9     0.7625    0.7077    0.7341      2169
       287.5     0.7588    0.7426    0.7506      1682
       305.1     0.8283    0.6503    0.7286      1224
         311     0.8015    0.7116    0.7539      1328
       401.9     0.8533    0.7619    0.8050      5657
      403.90     0.8388    0.7827    0.8098      1795
      403.91     0.7836    0.7819    0.7828       917
      410.71     0.6566    

In [35]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.8760500809121976
RUC AUC Score (Macro) = 0.8467306311494159


In [36]:
dev_df['prediction'] = dev_out
dev_df['tar'] = dev_tar

In [37]:
dev_df

,id,text,labels,prediction,tar
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753, 0.020503595, 0.03...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
15303,181850,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.002601429, 0.012164262, 0.087912336, 0.0511...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


### Avg outputs

In [38]:
out_mean_dict = dev_df.groupby('id').prediction.apply(np.mean).to_dict()
dev_df['out_mean'] = dev_df['id'].map(out_mean_dict)
dev_df

,id,text,labels,prediction,tar,out_mean
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00..."
...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753, 0.020503595, 0.03...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.013895996, 0.008231505, 0.070552155, 0.0524..."
15303,181850,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.002601429, 0.012164262, 0.087912336, 0.0511...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.015536042, 0.011225645, 0.21567202, 0.09861..."


In [39]:
df_mean = dev_df.drop_duplicates('id')

In [40]:
loss_mean = [nn.BCELoss()(torch.tensor(df_mean['out_mean'][i]), torch.tensor(df_mean['tar'][i])) for i in df_mean.index]
np.mean(loss_mean)

0.19554064

In [41]:
out_mean = np.vstack([df_mean['out_mean'][i]>=0.5 for i in df_mean.index])
targets = np.vstack([df_mean['tar'][i] for i in df_mean.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_mean)
f1_score_micro = metrics.f1_score(targets, out_mean, average='micro')
f1_score_macro = metrics.f1_score(targets, out_mean, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.24349270814103746
F1 Score (Micro) = 0.6235308521057787
F1 Score (Macro) = 0.5671806258890735


In [42]:
print(classification_report(targets, out_mean, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.5839    0.4974    0.5371       189
       038.9     0.6435    0.5423    0.5886       496
       244.9     0.7307    0.6217    0.6718       563
      250.00     0.7535    0.5426    0.6309       997
       272.0     0.7481    0.3620    0.4879       558
       272.4     0.7824    0.5557    0.6498      1087
       276.0     0.4156    0.6176    0.4968       319
       276.1     0.6890    0.4197    0.5216       417
       276.2     0.6767    0.4941    0.5711       589
       285.1     0.5559    0.6767    0.6104       566
       285.9     0.5902    0.4438    0.5066       649
       287.5     0.5701    0.4506    0.5034       415
       305.1     0.6917    0.4444    0.5412       414
         311     0.6688    0.4643    0.5481       448
       401.9     0.7951    0.6745    0.7298      2215
      403.90     0.7524    0.6200    0.6798       500
      403.91     0.7107    0.6589    0.6838       302
      410.71     0.4593    

In [43]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_mean, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_mean, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7855634536015719
RUC AUC Score (Macro) = 0.7692393782518829


### Most freq 5 labels

In [44]:
out_sum_dict = dev_df.groupby('id').prediction.apply(np.sum).to_dict()
dev_df['out_sum'] = dev_df['id'].map(out_sum_dict)
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540...","[13.620787, 12.88974, 0.1104764, 0.14175564, 0..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,...","[0.205751, 1.3536766, 0.8867351, 0.21622278, 0..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.03185281, 0.048090495, 0.11714831, 0.637938..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119...","[0.015514826, 0.094725385, 0.058641367, 0.0238..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00...","[0.020137992, 0.02387013, 0.00030884932, 0.001..."
...,...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0.11525594, 0.98818195, 0.21242945, 0.6646066..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009...","[0.121422835, 0.73317844, 0.29361296, 0.716036..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753, 0.020503595, 0.03...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.013895996, 0.008231505, 0.070552155, 0.0524...","[0.027791992, 0.01646301, 0.14110431, 0.104954..."
15303,181850,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.002601429, 0.012164262, 0.087912336, 0.0511...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.015536042, 0.011225645, 0.21567202, 0.09861...","[0.031072084, 0.02245129, 0.43134403, 0.197229..."


In [45]:
def freq_n(df, column): # column: out_sum
    df['freq_n'] = df[column]
    for idx in df.index:
      sorted = np.sort(df[column][idx])
      thres = sorted[-5] # position 5
      df['freq_n'][idx] = df[column][idx]>= thres


In [46]:
freq_n(dev_df, 'out_sum')

In [47]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_n
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540...","[13.620787, 12.88974, 0.1104764, 0.14175564, 0...","[True, False, False, False, False, False, Fals..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,...","[0.205751, 1.3536766, 0.8867351, 0.21622278, 0...","[False, True, False, False, False, False, Fals..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[False, False, False, True, False, False, Fals..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119...","[0.015514826, 0.094725385, 0.058641367, 0.0238...","[False, False, False, False, False, False, Fal..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00...","[0.020137992, 0.02387013, 0.00030884932, 0.001...","[False, False, False, False, False, False, Fal..."
...,...,...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0.11525594, 0.98818195, 0.21242945, 0.6646066...","[False, False, False, False, False, False, Tru..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009...","[0.121422835, 0.73317844, 0.29361296, 0.716036...","[False, False, False, False, False, False, Fal..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753, 0.020503595, 0.03...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.013895996, 0.008231505, 0.070552155, 0.0524...","[0.027791992, 0.01646301, 0.14110431, 0.104954...","[False, False, False, False, False, False, Tru..."
15303,181850,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.002601429, 0.012164262, 0.087912336, 0.0511...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.015536042, 0.011225645, 0.21567202, 0.09861...","[0.031072084, 0.02245129, 0.43134403, 0.197229...","[False, False, False, False, False, False, Fal..."


In [48]:
df_freq_n = dev_df.drop_duplicates('id')

In [49]:
out_freq_n = np.vstack([df_freq_n['freq_n'][i] for i in df_freq_n.index])
targets = np.vstack([df_freq_n['tar'][i] for i in df_freq_n.index])
#targets = dev_tar

f1_score_micro = metrics.f1_score(targets, out_freq_n, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq_n, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.5382733042746808
F1 Score (Macro) = 0.47133441027096173


In [50]:
print(classification_report(targets, out_freq_n, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.4127    0.4127    0.4127       189
       038.9     0.5520    0.4496    0.4956       496
       244.9     0.5952    0.5719    0.5833       563
      250.00     0.6511    0.5165    0.5761       997
       272.0     0.6271    0.3405    0.4413       558
       272.4     0.6688    0.4867    0.5634      1087
       276.0     0.2939    0.5925    0.3929       319
       276.1     0.4078    0.3765    0.3915       417
       276.2     0.4618    0.4414    0.4514       589
       285.1     0.4344    0.6731    0.5281       566
       285.9     0.3982    0.4037    0.4009       649
       287.5     0.3817    0.3928    0.3872       415
       305.1     0.4662    0.4493    0.4576       414
         311     0.4836    0.4286    0.4544       448
       401.9     0.7036    0.6772    0.6901      2215
      403.90     0.6783    0.5060    0.5796       500
      403.91     0.6367    0.6325    0.6346       302
      410.71     0.3775    

In [51]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq_n, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq_n, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7525935513504662
RUC AUC Score (Macro) = 0.7193712135831376


### Predicted percentage

In [52]:
note_count_dict = dev_df.groupby('id').size().to_dict()
dev_df['note_count'] = dev_df['id'].map(note_count_dict)

In [53]:
dev_df['out_bool'] = [(dev_df['prediction'][i]>=0.5).astype(int) for i in dev_df.index]

In [54]:

out_freq_dict = dev_df.groupby('id').out_bool.apply(np.sum).to_dict()
dev_df['num_pred'] = dev_df['id'].map(out_freq_dict)
dev_df['num_pred'] = [(dev_df['num_pred'][i]>=0.4*dev_df['note_count'][i]).astype(int) for i in dev_df.index]

In [55]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_n,note_count,out_bool,num_pred
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540...","[13.620787, 12.88974, 0.1104764, 0.14175564, 0...","[True, False, False, False, False, False, Fals...",14,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,...","[0.205751, 1.3536766, 0.8867351, 0.21622278, 0...","[False, True, False, False, False, False, Fals...",2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[False, False, False, True, False, False, Fals...",1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119...","[0.015514826, 0.094725385, 0.058641367, 0.0238...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00...","[0.020137992, 0.02387013, 0.00030884932, 0.001...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0.11525594, 0.98818195, 0.21242945, 0.6646066...","[False, False, False, False, False, False, Tru...",2,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009...","[0.121422835, 0.73317844, 0.29361296, 0.716036...","[False, False, False, False, False, False, Fal...",4,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753, 0.020503595, 0.03...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.013895996, 0.008231505, 0.070552155, 0.0524...","[0.027791992, 0.01646301, 0.14110431, 0.104954...","[False, False, False, False, False, False, Tru...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, ..."
15303,181850,admission date discharge d

In [56]:
df_freq = dev_df.drop_duplicates('id')

In [57]:
out_freq = np.vstack([df_freq['num_pred'][i] for i in df_freq.index])
targets = np.vstack([df_freq['tar'][i] for i in df_freq.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq)
f1_score_micro = metrics.f1_score(targets, out_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.21746354070518736
F1 Score (Micro) = 0.6137448905527754
F1 Score (Macro) = 0.5653857667844892


In [58]:
print(classification_report(targets, out_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.5076    0.5291    0.5181       189
       038.9     0.5740    0.5867    0.5803       496
       244.9     0.6679    0.6501    0.6589       563
      250.00     0.6979    0.5747    0.6304       997
       272.0     0.7318    0.3961    0.5140       558
       272.4     0.7413    0.5879    0.6557      1087
       276.0     0.3530    0.6552    0.4588       319
       276.1     0.6225    0.4508    0.5229       417
       276.2     0.6081    0.5110    0.5554       589
       285.1     0.5083    0.7049    0.5907       566
       285.9     0.5340    0.4715    0.5008       649
       287.5     0.4825    0.4651    0.4736       415
       305.1     0.6634    0.4855    0.5607       414
         311     0.6005    0.4933    0.5417       448
       401.9     0.7587    0.7084    0.7327      2215
      403.90     0.6911    0.6400    0.6646       500
      403.91     0.6491    0.6921    0.6699       302
      410.71     0.4320    

In [59]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7959551762141533
RUC AUC Score (Macro) = 0.7822716125849074


### Exponential moving average

In [60]:
def ewma(sub_df, window=3):
#    print(sub_df)
    alpha = 2 / (window + 1)
#    print(sub_df['ewma'])
    sub_df['ewma'] = sub_df['out_bool']
    for r in range(len(sub_df)):
        if r == 0:
            sub_df['ewma'].iloc[r] = sub_df['prediction'].iloc[r]
        else:
            sub_df['ewma'].iloc[r] = alpha*sub_df['prediction'].iloc[r] + (1-alpha)*sub_df['prediction'].iloc[r-1]
 #   print(type(sub_df['ewma']))
    return sub_df['ewma']

In [61]:

out_ewma_dict = dev_df.groupby('id', group_keys=False).apply(ewma).to_dict()
dev_df['out_ewma'] = pd.Series(dev_df.index, index=dev_df.index).map(out_ewma_dict)



In [62]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_n,note_count,out_bool,num_pred,out_ewma
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540...","[13.620787, 12.88974, 0.1104764, 0.14175564, 0...","[True, False, False, False, False, False, Fals...",14,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,...","[0.205751, 1.3536766, 0.8867351, 0.21622278, 0...","[False, True, False, False, False, False, Fals...",2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[False, False, False, True, False, False, Fals...",1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119...","[0.015514826, 0.094725385, 0.058641367, 0.0238...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00...","[0.020137992, 0.02387013, 0.00030884932, 0.001...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0.11525594, 0.98818195, 0.21242945, 0.6646066...","[False, False, False, False, False, False, Tru...",2,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009...","[0.121422835, 0.73317844, 0.29361296, 0.716036...","[False, False, False, False, False, False, Fal...",4,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.030593418, 0.2347742, 0.10243661, 0.2303535..."
15302,149837,admission date discharge date date of birth se...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0046994025, 0.0003621753

In [63]:
df_ewma = dev_df.drop_duplicates('id')

In [64]:
out_ewma = np.vstack([df_ewma['out_ewma'][i]>0.5 for i in df_ewma.index])
targets = np.vstack([df_ewma['tar'][i] for i in df_ewma.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_ewma)
f1_score_micro = metrics.f1_score(targets, out_ewma, average='micro')
f1_score_macro = metrics.f1_score(targets, out_ewma, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.5650908801020408
F1 Score (Macro) = 0.5196824823609691


In [65]:
print(classification_report(targets, out_ewma, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.4530    0.4339    0.4432       189
       038.9     0.5441    0.5101    0.5265       496
       244.9     0.6298    0.5258    0.5731       563
      250.00     0.6727    0.4865    0.5646       997
       272.0     0.6822    0.3154    0.4314       558
       272.4     0.7202    0.4995    0.5899      1087
       276.0     0.3235    0.5831    0.4161       319
       276.1     0.5534    0.3477    0.4271       417
       276.2     0.5626    0.4499    0.5000       589
       285.1     0.4882    0.6201    0.5463       566
       285.9     0.4831    0.3975    0.4362       649
       287.5     0.4541    0.4169    0.4347       415
       305.1     0.6418    0.4155    0.5044       414
         311     0.5778    0.4308    0.4936       448
       401.9     0.7423    0.6217    0.6767      2215
      403.90     0.6604    0.5640    0.6084       500
      403.91     0.6262    0.6325    0.6293       302
      410.71     0.4169    

In [66]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_ewma, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_ewma, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7590974847474276
RUC AUC Score (Macro) = 0.7489716038668577


### Most frequent prediction

In [67]:

most_freq_dict = dev_df.groupby('id')['out_bool'].apply(lambda x: x.value_counts().index[0]).to_dict()
dev_df['most_freq'] = dev_df['id'].map(most_freq_dict)

In [68]:
dev_df

,id,text,labels,prediction,tar,out_mean,out_sum,freq_n,note_count,out_bool,num_pred,out_ewma,most_freq
0,193230,age over year old female who presented to an o...,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[0.9729133, 0.9206957, 0.007891172, 0.01012540...","[13.620787, 12.88974, 0.1104764, 0.14175564, 0...","[True, False, False, False, False, False, Fals...",14,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.9718194, 0.9716812, 0.006728678, 0.00156904...","[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,179148,patient test information indication coronary a...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.1028755, 0.6768383, 0.44336754, 0.10811139,...","[0.205751, 1.3536766, 0.8867351, 0.21622278, 0...","[False, True, False, False, False, False, Fals...",2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.047537625, 0.52457875, 0.33284816, 0.062317...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,182444,pm chest port line placement clip clip number ...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[False, False, False, True, False, False, Fals...",1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0.03185281, 0.048090495, 0.11714831, 0.637938...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,170796,patient test information indication aortic dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.007757413, 0.047362693, 0.029320683, 0.0119...","[0.015514826, 0.094725385, 0.058641367, 0.0238...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.01066626, 0.092265606, 0.052056845, 0.01700...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,132541,npn resp infant remains orally intubated on se...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.010068996, 0.011935065, 0.00015442466, 0.00...","[0.020137992, 0.02387013, 0.00030884932, 0.001...","[False, False, False, False, False, False, Fal...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.016567651, 0.019664627, 0.00019598934, 0.00...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15300,147257,admission date discharge date date of birth se...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[0.10789406, 0.74788827, 0.18416962, 0.6372486...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0.11525594, 0.98818195, 0.21242945, 0.6646066...","[False, False, False, False, False, False, Tru...",2,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ...","[0.05762797, 0.49409097, 0.106214724, 0.332303...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, ..."
15301,107543,admission date discharge date date of birth se...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0.035818897, 0.46247655, 0.1968409, 0.2581515...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.030355709, 0.18329461, 0.07340324, 0.179009...","[0.121422835, 0.73317844, 0.29361296, 0.716036...","[False, False, False, Fals

In [69]:
df_most_freq = dev_df.drop_duplicates('id')

In [70]:
out_most_freq = np.vstack([df_most_freq['most_freq'][i] for i in df_most_freq.index])
targets = np.vstack([df_most_freq['tar'][i] for i in df_most_freq.index])
#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_most_freq)
f1_score_micro = metrics.f1_score(targets, out_most_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_most_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.580664400310133
F1 Score (Macro) = 0.5335732172341684


In [71]:
print(classification_report(targets, out_most_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.4919    0.4815    0.4866       189
       038.9     0.5761    0.5343    0.5544       496
       244.9     0.6527    0.5542    0.5994       563
      250.00     0.6889    0.5085    0.5851       997
       272.0     0.7063    0.3405    0.4595       558
       272.4     0.7305    0.5262    0.6118      1087
       276.0     0.3209    0.5674    0.4100       319
       276.1     0.6081    0.3981    0.4812       417
       276.2     0.5872    0.4686    0.5212       589
       285.1     0.4986    0.6449    0.5624       566
       285.9     0.5121    0.4222    0.4628       649
       287.5     0.4665    0.4193    0.4416       415
       305.1     0.6486    0.4324    0.5188       414
         311     0.5952    0.4464    0.5102       448
       401.9     0.7571    0.6415    0.6945      2215
      403.90     0.6636    0.5800    0.6190       500
      403.91     0.6471    0.6556    0.6513       302
      410.71     0.4258    

In [72]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_most_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_most_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7681767347010525
RUC AUC Score (Macro) = 0.7562584555586571


### Testing

In [73]:
# Evaluate the model

# Evaluate the model

def testing():
    model.eval()

    fin_targets=[]
    fin_outputs=[]
    losses=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            fin_targets.extend(targets.cpu().detach().numpy())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy())
    print(f'Loss:  {np.mean(losses):.2f}')
    return fin_outputs, fin_targets, losses

In [74]:
test_out, targets, losses = testing()
outputs = np.array(test_out) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Loss:  0.14
F1 Score (Micro) = 0.7883350066543697
F1 Score (Macro) = 0.7179666182679436


In [75]:
print(classification_report(targets, outputs, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.7508    0.7585    0.7546       969
       038.9     0.8216    0.8031    0.8123      2971
       244.9     0.8309    0.7882    0.8090      2469
      250.00     0.8597    0.7303    0.7897      3674
       272.0     0.8769    0.5886    0.7044      1682
       272.4     0.8677    0.7305    0.7932      3770
       276.0     0.6781    0.8519    0.7552      2337
       276.1     0.8480    0.7349    0.7874      1973
       276.2     0.8202    0.7749    0.7969      3168
       285.1     0.7081    0.8153    0.7579      2452
       285.9     0.7665    0.7097    0.7370      2701
       287.5     0.7605    0.7684    0.7644      2107
       305.1     0.8390    0.6371    0.7242      1521
         311     0.8321    0.7311    0.7783      1640
       401.9     0.8579    0.7660    0.8094      7163
      403.90     0.8360    0.7796    0.8068      2210
      403.91     0.7732    0.7505    0.7616      1090
      410.71     0.6793    

In [76]:

ruc_auc_score_micro = metrics.roc_auc_score(targets, outputs, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, outputs, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.87917404489835
RUC AUC Score (Macro) = 0.84889039689001


In [77]:

test_df['prediction'] = test_out
test_df['tar'] = targets

In [78]:
note_count_dict = test_df.groupby('id').size().to_dict()
test_df['note_count'] = test_df['id'].map(note_count_dict)

In [79]:
test_df['out_bool'] = [(test_df['prediction'][i]>=0.5).astype(int) for i in test_df.index]

In [80]:

out_freq_dict = test_df.groupby('id').out_bool.apply(np.sum).to_dict()
test_df['num_pred'] = test_df['id'].map(out_freq_dict)
test_df['num_pred'] = [(test_df['num_pred'][i]>=0.4*test_df['note_count'][i]).astype(int) for i in test_df.index]

In [81]:
df_freq = test_df.drop_duplicates('id')

In [82]:
out_freq = np.vstack([df_freq['num_pred'][i] for i in df_freq.index])
targets = np.vstack([df_freq['tar'][i] for i in df_freq.index])

#targets = dev_tar
accuracy = metrics.accuracy_score(targets, out_freq)
f1_score_micro = metrics.f1_score(targets, out_freq, average='micro')
f1_score_macro = metrics.f1_score(targets, out_freq, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

F1 Score (Micro) = 0.6156882608771592
F1 Score (Macro) = 0.5693118086017995


In [83]:
print(classification_report(targets, out_freq, target_names=icd_classes_50, digits=4))

              precision    recall  f1-score   support

      008.45     0.4951    0.5050    0.5000       202
       038.9     0.5488    0.5623    0.5555       530
       244.9     0.6531    0.6105    0.6311       629
      250.00     0.7223    0.5806    0.6437      1111
       272.0     0.7524    0.3816    0.5064       621
       272.4     0.7503    0.5807    0.6547      1221
       276.0     0.3488    0.6374    0.4509       353
       276.1     0.6383    0.4626    0.5364       454
       276.2     0.5907    0.4976    0.5402       635
       285.1     0.4735    0.6719    0.5555       637
       285.9     0.5352    0.4523    0.4903       723
       287.5     0.4954    0.4853    0.4903       441
       305.1     0.6844    0.4710    0.5580       465
         311     0.6546    0.5370    0.5900       473
       401.9     0.7689    0.7013    0.7335      2471
      403.90     0.6667    0.6216    0.6433       547
      403.91     0.6380    0.6420    0.6400       324
      410.71     0.4711    

In [84]:
ruc_auc_score_micro = metrics.roc_auc_score(targets, out_freq, average='micro')
ruc_auc_score_macro = metrics.roc_auc_score(targets, out_freq, average='macro')

print(f"RUC AUC Score (Micro) = {ruc_auc_score_micro}")
print(f"RUC AUC Score (Macro) = {ruc_auc_score_macro}")

RUC AUC Score (Micro) = 0.7964281381767501
RUC AUC Score (Macro) = 0.781669443510269
